In [ ]:
from pyrosm import OSM
import geopandas as gpd
import pandas as pd
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, Polygon, MultiPoint
import folium
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Patch
from matplotlib.lines import Line2D
import alphashape
import contextily 
from descartes import PolygonPatch
from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap


In [ ]:
# assign file path to osm pbf file.
base_road_path = 'C:/Users/hular/projects/ClosestDestination/testEnvironment/Data/belfast_slightly_trimmed.osm.pbf'

osm = OSM(base_road_path)

#load road network from graph
G_network = osm.get_network(network_type = 'driving', nodes = True)

# #Convert to ga graph (this will create if not a pyrosm source)
# if not isinstance(G, nx.Graph):
#     G = nx.from_pandas_edgelist(G, source='u', target='v', edge_attr=True)

nodes, edges = osm.get_network(network_type='driving', nodes=True)

#generate graph of network
G = osm.to_graph(nodes, edges, graph_type='networkx')

edges.plot(figsize=(10,10))


In [ ]:
#load in the data which service areas will be analysed.
#sticking it in a try loop forces it to work.
service_locations_csv = pd.read_csv('C:/Users/hular/projects/ClosestDestination/testEnvironment/Data/libraries_belfast_2024_very_trimmed.csv')

def csv_to_gdf(csv = service_locations_csv, x = 'X COORDINATE', y = 'Y COORDINATE', input_crs = 29902, crs_conversion = None):
    """ function to convert csv to a gdf based off X, Y coordinates and input CRS. Optional CRS conversion.
    
        Parameters:
    - csv: DataFrame, source data.
    - x: str, column name for the x coordinate.
    - y: str, column name for the y coordinate.
    - input_crs: int, EPSG code for input coordinate reference system.
    - crs_conversion: int, optional EPSG code for converting CRS."""
    #create a list for each row of geom by zipping and turn into a point tuple
    try:
        csv['geometry'] = list(zip(csv[x], csv[y]))
        csv['geometry'] = csv['geometry'].apply(Point)
        # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame(csv, geometry='geometry', crs=f'EPSG:{input_crs}')
        #Only converts if specified
        if crs_conversion:
            gdf = gdf.to_crs(epsg=crs_conversion)
        return gdf
    except Exception:
        print(f'Exception error: {Exception}')
        
        # Return the GeoDataFrame

service_locations = csv_to_gdf(csv=service_locations_csv, crs_conversion=4326)
service_locations.head(25)

In [ ]:

#Function to create a dict of names and nearest names to be called in a loop creating the subgraphs for each location.
def nearest_node_and_name(Network_graph, service_locations,  col_name):
    """ Create a dictionary of location names and nearest node on Graph  """   
    service_xy = {}
    
    for index, row in service_locations.iterrows():
        #extract x and y for each library
        location_x = row['geometry'].x
        location_y = row['geometry'].y
        #calculate the nearest node on the Graph
        nearest_node = ox.distance.nearest_nodes(Network_graph, location_x, location_y)
        # extract the library name
        name = row[col_name]
        #Combine the name and nearest name. 
        service_xy[name] = {'nearest_node': nearest_node}
    return service_xy

service_locations_nearest_node = nearest_node_and_name(G, service_locations, col_name = 'Static Library Name')


In [ ]:
def difference_poly(gdf, category):
    """ This will create symmetrical difference of POLYGONS based on different categories """


#Function to iterate through star locations. Input must be from a dict/array e.g. Ardoyne Library: POLYGON((-5.49, 54.6),(-5.55, 54.4))
#Next step to iterate over cutoff list. e.g. [1000, 2000,3000,4000]
def single_source_polygon(nearest_node_dict, graph = G, cutoffs = [1000,2000,3000], alpha_value = 500, weight='length'):
    """ Returns a GeoPandas Dataframe of service area polygons"""
    # service_areas_dict = {} #uncomment with services_ares_dict[name]
    data_for_gdf = []

    #For each start location [name] creates a polygon around the point.
    for name in nearest_node_dict:

        #cycle through each distance in list supplied
        for distance in cutoffs:
          
            #Extract nearest node to the name (start location)
            nearest_node = nearest_node_dict[name]['nearest_node']
            subgraph = nx.single_source_dijkstra_path_length(graph, nearest_node, cutoff=distance, weight = weight)
            
            #Creates a list of all nodes which are reachable within the cutoff.
            reachable_nodes = list(subgraph.keys())
            node_points_list = []
            for node in reachable_nodes:
                x = graph.nodes[node]['x']
                y = graph.nodes[node]['y']
                node_points_list.append(Point(x, y))

            # Makes the x,y values into just a list of tuples to be used with alpha shape
            node_points_series = pd.Series(node_points_list)
            node_point_series_tuples_list = node_points_series.apply(lambda point: (point.x, point.y))
            correct_points_list = node_point_series_tuples_list.tolist()
            
            #Create an alpha shape for each polygon.
            alpha_shape = alphashape.alphashape(correct_points_list, alpha_value)
        
            data_for_gdf.append({'name': name, 'distance':distance, 'geometry': alpha_shape})
            # service_areas_dict[name] = alpha_shape #uncomment to check if function returns correct variables

    gdf_alpha = gpd.GeoDataFrame(data_for_gdf, crs= 4326)
    
        
    #return the geodataframe
    return gdf_alpha
alpha_areas = single_source_polygon(nearest_node_dict= service_locations_nearest_node, graph = G, cutoffs=[1000,2000,3000], alpha_value=500, weight='length')
 
# Uncomment pprint to check output is correct.
# import pprint
# pprint.pprint(alpha_areas) 

In [ ]:
#Dissolve distance cutoff polygons from alpha_areas, create 'contours' by dissolving and difference
#Mostly for visualisation purposes, however can be for some basic analysis. 
#This doesn't retain information on the closest library as all polygons dissolved

#dissolving first gives loads of errors NaN/infinite errors when differencing. First subset -> dissolve -> difference
alpha_1km = alpha_areas[alpha_areas['distance'] == 1000].reset_index()
alpha_2km = alpha_areas[alpha_areas['distance'] == 2000].reset_index()
alpha_3km = alpha_areas[alpha_areas['distance'] == 3000].reset_index()
#dissolve filtered data
alpha_3km_dissolve = alpha_3km.dissolve()
alpha_2km_dissolve = alpha_2km.dissolve()
alpha_1km_dissolve = alpha_1km.dissolve()
#get difference of all but smallest cutoff distance
alpha_dissolve_diff_3km = alpha_3km_dissolve.difference(alpha_2km_dissolve)
alpha_dissolve_diff_2km = alpha_2km_dissolve.difference(alpha_1km_dissolve)

#plot the service areas, i don't know why the areas don't show on legend.
fig, ax = plt.subplots(figsize=(15, 15))

alpha_dissolve_diff_3km.plot(ax=ax, color='Red', alpha=0.5, label = '3km')
alpha_dissolve_diff_2km.plot(ax=ax, color='Orange', alpha=0.5, label = '2km')
alpha_1km_dissolve.plot(ax=ax, color = 'Green', alpha = 0.5, label = '1km')
edges.plot(ax=ax, color='black', linewidth=0.3, label='Road Network',zorder=1)
service_locations.plot(ax=ax, color='purple', markersize=20, label='Libraries',zorder=1)
#legend handles
legend_elements = [
    Patch(facecolor='Red', edgecolor='Red', label='3km', alpha=0.5),
    Patch(facecolor='Orange', edgecolor='Orange', label='2km', alpha=0.5),
    Patch(facecolor='Green', edgecolor='Green', label='1km', alpha=0.5),
    Line2D([0], [0], color='black', label='Road Network', linewidth=1), 
    Line2D([0], [0], marker='o', color='w', label='Libraries', markersize=10, markerfacecolor='purple')
]
ax.legend(handles=legend_elements, title="Legend")

plt.autoscale(enable=True, axis='both', tight=True)
plt.show()

In [ ]:
# #Example single not looped yet.
# #loop over library nearest node dict with single_source_path_length, 
# # create a new dict with name and polygons created from the reachable nodes and retain the travel distance.
# #Create closest node for example. Loop this later.
# single_location = service_locations.iloc[1]
# x,y =(single_location['geometry'].x, single_location['geometry'].y)

# nearest_node = ox.nearest_nodes(G, X=x, Y=y )

# # distance in metres, time = seconds. For creation of subgraph
# max_travel_distance = 1000

# # Create a sub-graph and a list of nodes within the specified distance.
# #subgraph = subsection of the main Graph based on cutoff
# subgraph = nx.single_source_dijkstra_path_length(G, nearest_node, cutoff=max_travel_distance, weight='length')
# reachable_nodes = list(subgraph.keys())

# node_points = [Point(G.nodes[node]['x'], G.nodes[node]['y']) for node in reachable_nodes]
# polygon = Polygon([[point.x, point.y] for point in node_points])
# print(polygon)

# # Create a gdf for the polygon.
# import geopandas as gpd
# gdf = gpd.GeoDataFrame([polygon], columns=['geometry'], crs='EPSG:4326')

# fig, ax = plt.subplots()
# gdf.boundary.plot(ax=ax)
# gdf.plot(ax=ax, color='blue', alpha=0.5)
# plt.show()



# service_point = (54.55998716608919, -5.983139503268679)  # Latitude, Longitude

# m = folium.Map(location=[service_point[0], service_point[1]], zoom_start=14)

# folium.GeoJson(gdf).add_to(m)

# m.save(r"C:\Users\hular\projects\ClosestDestination\testEnvironment\service_area_map.html")